In [ ]:
import mne
import os

BASE_DIR = r"rEEG"
SUBJ = "sub-121"

raw_path = os.path.join(BASE_DIR, SUBJ, "postICA121_raw.fif")
raw = mne.io.read_raw_fif(raw_path, preload=True)

# 2-second epochs with 50% overlap
events = mne.make_fixed_length_events(raw, duration=2.0, overlap=1.0)
reject_criteria = dict(eeg=200e-6)  # relaxed threshold (200 µV)

epochs = mne.Epochs(raw, events=events, tmin=0, tmax=2.0,
                    baseline=None, reject=reject_criteria, preload=True)

# Remove subjects with <20 clean epochs
if len(epochs) < 20:
    print(f"{SUBJ} has too few clean epochs ({len(epochs)}), skipping.")
else:
    EPOCH_OUT_NAME = "epo_121_raw.fif"
    epochs.save(os.path.join(BASE_DIR, SUBJ, EPOCH_OUT_NAME), overwrite=True)
    print(f"{SUBJ}: {len(epochs)} clean epochs saved → {EPOCH_OUT_NAME}")


Opening raw data file rEEG\sub-078\postICA078_raw.fif...
    Range : 0 ... 36464 =      0.000 ...   145.856 secs
Ready.
Reading 0 ... 36464  =      0.000 ...   145.856 secs...
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 144 events and 501 original time points ...
0 bad epochs dropped
sub-078: 144 clean epochs saved → epo_059_raw.fif


C:\Users\User\AppData\Local\Temp\ipykernel_42796\721354534.py:22: RuntimeWarning: This filename (rEEG\sub-078\epo_059_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(os.path.join(BASE_DIR, SUBJ, EPOCH_OUT_NAME), overwrite=True)


In [3]:
import mne
import os
import numpy as np

# -------------------------
# CONFIG
# -------------------------
BASE_DIR = r"rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(111,112)]  # adjust range
EPOCH_DURATION = 4.0  # seconds
OVERLAP = 0.5         # 50%
REJECT_THRESHOLD = dict(eeg=175e-6)  # 150 µV for EEG channels
MIN_EPOCHS = 20

RAW_NAME = "postICA{sid}_raw.fif"
EPOCH_OUT_NAME = "epo_{sid}_raw.fif"

# -------------------------
# LOOP OVER SUBJECTS
# -------------------------
for subj in SUBJECTS:
    sid = subj.split("-")[1]
    print(f"\n=== Processing {subj} ===")

    raw_path = os.path.join(BASE_DIR, subj,'eeg', RAW_NAME.format(sid=sid))
    out_path = os.path.join(BASE_DIR, subj, EPOCH_OUT_NAME.format(sid=sid))

    if not os.path.exists(raw_path):
        print("Missing raw file — skipping")
        continue

    # Load cleaned raw
    raw = mne.io.read_raw_fif(raw_path, preload=True)

    # Create overlapping events for continuous data
    step = EPOCH_DURATION * (1 - OVERLAP)
    n_steps = int((raw.times[-1] - EPOCH_DURATION) // step) + 1
    events = []
    for i in range(n_steps):
        sample = int(i * step * raw.info['sfreq'])
        events.append([sample, 0, 1])
    events = np.array(events)

    # Epoching
    epochs = mne.Epochs(
        raw, events=events, tmin=0, tmax=EPOCH_DURATION,
        baseline=None, preload=True, reject=REJECT_THRESHOLD
    )

    # Check minimum epochs
    if len(epochs) < MIN_EPOCHS:
        print(f"Subject {subj} has only {len(epochs)} clean epochs — skipping")
        continue

    # Save epochs
    epochs.save(out_path, overwrite=True)
    print(f"{subj}: {len(epochs)} clean epochs saved → {out_path}")

print("\n=== ALL SUBJECTS PROCESSED ===")



=== Processing sub-111 ===
Missing raw file — skipping

=== ALL SUBJECTS PROCESSED ===
